In [1]:
import pandas as pd
import dspy
from dspy.evaluate import SemanticF1
import json
import requests
import boto3

In [2]:
# Helper functions to setup datset and evaluation
def create_dataset(path: str) -> list[dspy.Example]:
	dataset = pd.read_excel(path)
	dataset_dict = dataset.to_dict(orient='records')
	dspy_dataset = []

	for row in dataset_dict:
		dspy_dataset.append(dspy.Example(question=row['input'], response=row['expected_output']).with_inputs("question"))

	print(f"Dataset length: {len(dspy_dataset)}")
	return dspy_dataset

def create_sets(dataset: list[dspy.Example], metric: dspy.Module = SemanticF1()):
	trainset, valset, devset, testset = dataset[:15], dataset[15:30], dataset[30:40], dataset[40:]
	evaluate = dspy.Evaluate(devset=devset, metric=metric, num_threads=24, display_progress=True, display_table=3)

	for name, set in zip([trainset, valset, devset, testset], ["trainset", "valset", "devset", "testset"]):
		print(f"{set} length: {len(name)}")

	return trainset, valset, devset, testset, evaluate

In [4]:
dspy_dataset = create_dataset("synthetics/synthetic_dataset_revised.xlsx")
trainset, valset, devset, testset, evaluate = create_sets(dspy_dataset)

# Save the testset to a file
with open("testset.json", "w") as f:
	json.dump([example.toDict() for example in testset], f)

Dataset length: 52
trainset length: 15
valset length: 15
devset length: 10
testset length: 12


In [5]:
def search(query: str, top_k: int) -> list[str]:
    url = "http://greencompute-1575332443.us-east-1.elb.amazonaws.com/api/llm/retrieval"
    headers = {
        "accept": "application/json",
        "Content-Type": "application/json"
    }
    data = {
        "query": query,
        "top_k": top_k
    }

    documents = requests.post(url, headers=headers, json=data).json()["documents"]
    return [f"[{i}]" + doc["doc_title"] + doc["url"] + "\n\n" + doc["content"] for i, doc in enumerate(documents)]

class TitanLM(dspy.LM):
    def __init__(self, model: str, client, max_tokens: int = 1024, temperature: float = 0.7, top_p: float = 0.9, **kwargs):
        self.client = client
        self.history = []
        self.temperature = temperature
        self.max_tokens = 1024
        self.top_p = top_p

        super().__init__(model, **kwargs)
        self.model = model
    
    def _format_message(self, prompt: str):
        body = json.dumps(
            {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": self.max_tokens,
                    "stopSequences": [],
                    "temperature": self.temperature,
                    "topP": self.top_p,
                },
            }
        )
        return body

    def generate_content(self, prompt: str) -> str:
        body = self._format_message(prompt)
        response = self.client.invoke_model(
            body=body,
            modelId=self.model,
            accept="application/json",
            contentType="application/json",
        )
        response_body = json.loads(response.get("body").read())
        return response_body.get("results")

    def __call__(self, prompt=None, messages=None, **kwargs):
        # Custom chat model working for text completion model
        prompt = '\n\n'.join([x['content'] for x in messages] + ['BEGIN RESPONSE:'])

        completions = self.generate_content(prompt)
        self.history.append({"prompt": prompt, "completions": completions})

        # Must return a list of strings
        return [completions[0].get("outputText")]

    def inspect_history(self):
        for interaction in self.history:
            print(f"Prompt: {interaction['prompt']} -> Completions: {interaction['completions']}")

In [6]:
class RAG(dspy.Module):
    def __init__(self, num_docs=20):
        self.num_docs = num_docs
        self.respond = dspy.ChainOfThought('context, question -> response')

    def forward(self, question):
        context = search(question, top_k=self.num_docs)
        return self.respond(context=context, question=question)

In [7]:
lm = TitanLM("amazon.titan-text-premier-v1:0", client=boto3.client("bedrock-runtime"))
dspy.configure(lm=lm)

In [8]:
rag = RAG()
rag(question="How do I increase my data center efficiency?")

Prediction(
    reasoning='The document discusses data center energy efficiency.',
    response='1. Perform an energy assessment.\n2. Understand the data center cooling system.\n3. Implement an energy management plan.\n4. Monitor and measure energy use.\n5. Upgrade to energy efficient equipment.'
)

In [9]:
evaluate(RAG())

Average Metric: 2.1749871991807477 / 10  (21.7): 100%|██████████| 10/10 [00:24<00:00,  2.49s/it]
2024/11/12 19:07:13 INFO dspy.evaluate.evaluate: Average Metric: 2.1749871991807477 / 10 (21.7%)


,question,example_response,reasoning,pred_response,SemanticF1
0,"What factors, including multi-tenancy and operator incentives, contribute to cloud's lower environmental impact?","Cloud services generally have a lower environmental impact due to several key factors: 1. **Multi-tenancy**: This concept allows multiple customers to share resources, leading to...",The context does not mention anything about cloud's lower environmental impact.,I cannot answer this question based on the given context.,
1,"Identify how virtualization, power management, and cloud computing reduce energy waste and enhance business operations in server rooms.","Virtualization reduces energy waste in server rooms by allowing multiple virtual machines to run on a single physical server, optimizing hardware usage and decreasing the...",I cannot find sufficient information to answer the question.,"Sorry, I cannot find sufficient information to answer the question.",
2,How do virtualization and cloud computing synergistically enhance server room energy efficiency and utilization rates?,Virtualization and cloud computing work together to significantly enhance server room energy efficiency and utilization rates by improving resource allocation and reducing energy consumption. Virtualization...,I cannot find sufficient information to answer the question.,"Sorry, I cannot find sufficient information to answer the question.",


21.75

In [11]:
metric = SemanticF1()
tp = dspy.MIPROv2(metric=metric, auto="medium", num_threads=24)  # use fewer threads if your rate limit is small

optimized_rag = tp.compile(
    RAG(), 
    trainset=trainset, 
    valset=valset,
	max_bootstrapped_demos=2,
 	max_labeled_demos=2,
  	requires_permission_to_run=False
)

2024/11/12 19:45:09 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING MEDIUM AUTO RUN SETTINGS:
num_trials: 25
minibatch: False
num_candidates: 19
valset size: 15

2024/11/12 19:45:09 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2024/11/12 19:45:09 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2024/11/12 19:45:09 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=19 sets of demonstrations...


Bootstrapping set 1/19
Bootstrapping set 2/19
Bootstrapping set 3/19


 87%|████████▋ | 13/15 [02:29<00:23, 11.53s/it]


Bootstrapped 2 full traces after 13 examples for up to 1 rounds, amounting to 13 attempts.
Bootstrapping set 4/19


 53%|█████▎    | 8/15 [01:09<01:01,  8.73s/it]


...

Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens, please wait before trying again.


  0%|          | 0/15 [00:00<?, ?it/s]2024/11/12 20:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 16'].
2024/11/12 20:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [21.82, 57.59, 69.31, 46.68, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2024/11/12 20:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 69.31
2024/11/12 20:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2024/11/12 20:19:00 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 25 / 25 =====


Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens, please wait before trying again.


  0%|          | 0/15 [00:00<?, ?it/s]2024/11/12 20:19:41 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 0.0 with parameters ['Predictor 0: Instruction 7', 'Predictor 0: Few-Shot Set 18'].
2024/11/12 20:19:41 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [21.82, 57.59, 69.31, 46.68, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
2024/11/12 20:19:41 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 69.31
2024/11/12 20:19:41 INFO dspy.teleprompt.mipro_optimizer_v2: =========================


2024/11/12 20:19:41 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 69.31!


Exception occurred: An error occurred (ThrottlingException) when calling the InvokeModel operation (reached max retries: 4): Too many tokens, please wait before trying again.


In [12]:
optimized_rag.save("optimized_rag_v2.json")